# Clasificación de Sentimientos de Críticas de Películas (utilizando Naive Bayes, Logistic Regression y n-gramas)

Este notebook es una adaptación del existente en el curso de Fast.ai de procesamiento del lenguaje natural https://github.com/fastai/course-nlp/blob/master/3-logreg-nb-imdb.ipynb

Se mostratán las siguientes técnicas de clasificación de sentimientos:

- Naive Bayes
- Logistic Regression
- ngrams

Se emplearán las siguientes librerías (no requieren instalación en colab): 

- *fastai* [the fastai library](https://docs.fast.ai) : Para instalarla utilizar `pip install -U scikit-learn`
- *sklearn* [the scikit-learn library](https://scikit-learn.org/stable/user_guide.html):  Para instalarla se aconseja utilizar: `conda install -c pytorch -c fastai fastai=1.0` ó 
`pip install fastai==1.0`


## IMDB dataset

Este dataset ([large movie review dataset](http://ai.stanford.edu/~amaas/data/sentiment/))  contiene una colección de 50.000 críticas de IMDB y Fast.ai aloja estos dataset en AWS [fast.ai datasets](https://course.fast.ai/datasets.html). 

En la versión de fastai, consideraron críticas que tienen una alta polarización:

- Una crítica negativa, es aquel que contiene un score ≤ 4/10
- Una crítica positiva es aquella que tiene un puntaje ≥ 7/10
- No se incluyeron críticas neutrales.


Usualmente el dataset - y en este caso se aplica -, está dividido en dos colecciones. 
- Una de entrenamiento *training* 
- Otra de pruebas *testing*

La tarea de **clasificación de sentimientos** consiste en predecir la polaridad (que tan positivo o que tan negativo es un texto dado).


### Imports

En esta parte se importarán las librerías a emplear en este notebook:

In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
from fastai import *
from fastai.text import *

In [3]:
import sklearn.feature_extraction.text as sklearn_text

### Tokenización y creación de la matriz de Documento-término

fast.ai tiene una colección de datasets [datasets hosted via AWS Open Datasets](https://course.fast.ai/datasets.html) para descarga rápida usando URLs. 


In [4]:
#?? URLs

Es una buena idea empezar con un dataset de ejemplo antes de utilizar el dataset completo para realizar cómputos rápidos y hacer que el modelo funcione. Así se puede obtener un dataset de ejemplo:

In [5]:
path = untar_data(URLs.IMDB_SAMPLE)
path

WindowsPath('C:/Users/Arles/.fastai/data/imdb_sample')

Es posible apreciar el contenido del dataframe usando pandas:

In [6]:
df = pd.read_csv(path/'texts.csv')
df.head()

,label,text,is_valid
0,negative,Un-bleeping-believable! Meg Ryan doesn't even ...,False
1,positive,This is a extremely well-made film. The acting...,False
2,negative,Every once in a long while a movie will come a...,False
3,positive,Name just says it all. I watched this movie wi...,False
4,negative,This movie succeeds at being one of the most u...,False


En este notebook se utilizará [TextList](https://docs.fast.ai/text.data.html#TextList) que se incluye en la librería fastai:

In [7]:
movie_reviews = (TextList.from_csv(path, 'texts.csv', cols='text')
                         .split_from_df(col=2)
                         .label_from_df(cols=0))

### Exploración de los datos

Un punto de inicio para cualquier problema que involucre datos es explorarlos y ver como lucen. En este caso vamos a análizar críticas de películas que han sido **etiquetadas** como **positivas** o **negativas.** 


In [8]:
movie_reviews.valid.x[0], movie_reviews.valid.y[0] 

(Text xxbos xxmaj this very funny xxmaj british comedy shows what might happen if a section of xxmaj london , in this case xxmaj xxunk , were to xxunk itself independent from the rest of the xxup uk and its laws , xxunk & post - war xxunk . xxmaj merry xxunk is what would happen . 
  
   xxmaj the explosion of a wartime bomb leads to the xxunk of ancient xxunk which show that xxmaj xxunk was xxunk to the xxmaj xxunk of xxmaj xxunk xxunk ago , a small historical xxunk long since forgotten . xxmaj to the new xxmaj xxunk , however , this is an unexpected opportunity to live as they please , free from any xxunk from xxmaj xxunk . 
  
   xxmaj stanley xxmaj xxunk is excellent as the minor city xxunk who suddenly finds himself leading one of the world 's xxunk xxunk . xxmaj xxunk xxmaj margaret xxmaj xxunk is a delight as the history professor who sides with xxmaj xxunk . xxmaj others in the stand - out cast include xxmaj xxunk xxmaj xxunk , xxmaj paul xxmaj xxunk , xxmaj xxunk xxmaj xxunk ,

En NLP, un **token** es la unidad básica de procesamiento (el concepto de token depende de la aplicación y de lo que se quiera realizar). En esta parte, un token hace referencia a palabras o a signos de puntuación y también tendremos algunos tokens especiales que corresponden a palabras desconocidas, mayúsculas, etc.  Los tokens que empiezan con "xx" son especiales y son generados por fast.ai. En la ([documentación de fast.ai](https://fastai1.fast.ai/text.transform.html#Tokenizer)) se puede apreciar una lista de tokens y sus significados: 

![](https://raw.githubusercontent.com/arleserp/CCE2021/main/images/tokens.png)


Podemos apreciar que el resultado del procesamiento en fast.ai de el dataset de ejemplo genera dos dataset uno de entrenamiento con 800 registros y otro de validación con 200:

In [9]:
len(movie_reviews.train.x), len(movie_reviews.valid.x)

(800, 200)

Adicionalmente se generan una lista y un diccionario. La lista incluye un mapeo de enteros a tokens `(movie_reviews.vocab.itos)` y el diccionario mapea de tokens a enteros (`movie_reviews.vocab.stoi`). ¿Por qué cree que son de diferentes tamaños?: 

In [10]:
print(type(movie_reviews.vocab.itos), type(movie_reviews.vocab.stoi))
len(movie_reviews.vocab.itos), len(movie_reviews.vocab.stoi)

<class 'list'> <class 'collections.defaultdict'>


(6008, 19161)

Si miramos que número de token tiene la palabra language:

In [11]:
movie_reviews.vocab.stoi['language'] 

917

Si miramos la palabra a la que corresponde el token 917:

In [12]:
movie_reviews.vocab.itos[917]

'language'

Las palabras de la 20 a la 29 son:

In [13]:
movie_reviews.vocab.itos[20:30]

['that', 'this', '"', "'s", '\n \n ', '-', 'was', 'as', 'for', 'movie']

La palabra 6007 es:

In [14]:
movie_reviews.vocab.itos[6007]

'sollett'

In [15]:
movie_reviews.vocab.itos[:20] #primeras 20 palabras

['xxunk',
 'xxpad',
 'xxbos',
 'xxeos',
 'xxfld',
 'xxmaj',
 'xxup',
 'xxrep',
 'xxwrep',
 'the',
 '.',
 ',',
 'and',
 'a',
 'of',
 'to',
 'is',
 'it',
 'in',
 'i']

In [16]:
movie_reviews.vocab.stoi

defaultdict(int,
            {'xxunk': 0,
             'xxpad': 1,
             'xxbos': 2,
             'xxeos': 3,
             'xxfld': 4,
             'xxmaj': 5,
             'xxup': 6,
             'xxrep': 7,
             'xxwrep': 8,
             'the': 9,
             '.': 10,
             ',': 11,
             'and': 12,
             'a': 13,
             'of': 14,
             'to': 15,
             'is': 16,
             'it': 17,
             'in': 18,
             'i': 19,
             'that': 20,
             'this': 21,
             '"': 22,
             "'s": 23,
             '\n \n ': 24,
             '-': 25,
             'was': 26,
             'as': 27,
             'for': 28,
             'movie': 29,
             'with': 30,
             'but': 31,
             'film': 32,
             'you': 33,
             ')': 34,
             'on': 35,
             '(': 36,
             "n't": 37,
             'are': 38,
             'he': 39,
             'his': 40,
       

Let's test that a non-word maps to xxunk:

In [17]:
movie_reviews.vocab.itos[movie_reviews.vocab.stoi['rrachell']]

'xxunk'

In [18]:
movie_reviews.vocab.itos[movie_reviews.vocab.stoi['language']]

'language'

In [19]:
t = movie_reviews.train[0][0]

In [20]:
t.data[:30]

array([   2,    5, 4619,   25,    0,   25,  867,   52,    5, 3776,    5, 1800,   95,   37,   85,  191,   64,  935,
          0, 2738,  517,   18,   21,   11,   84, 2417,  192,   88, 3777,   64], dtype=int64)

## Creación manual de nuestra matriz de término-documento

Esta matriz representa una bolsa de palabras. No se mantiene un registro de como las palabras están ordenadas, solamente cuales palabras ocurren y que tan a menudo.

Es posible utilizar [sklearn's CountVectorizer](https://github.com/scikit-learn/scikit-learn/blob/55bf5d9/sklearn/feature_extraction/text.py#L940). En este notebook se creará de forma manual para:
- entender lo que sklearn está haciendo por debajo
- crear algo que se integre con TextList de fastai


Para crear esta matriz es necesario entender un poco como funcionan los **Counter** de python y las matrices **dispersas**.


### Counters

Un counter permite contar la cantidad de ocurrencias de una lista.

In [21]:
c = Counter([4,2,8,8,4,8]) 

In [22]:
c

Counter({4: 2, 2: 1, 8: 3})

In [23]:
c.values()

dict_values([2, 1, 3])

In [24]:
c.keys()

dict_keys([4, 2, 8])

Counter pertenece al modulo de collections de python (como OrderedDict, defaultdict, deque, and namedtuple).

### Matrices disperas en Scipy (in Scipy)

Aunque se han reducido de 19.000 palabras a 6000. La mayoría de los token no aparecen en la mayoría de críticas. Esto nos permite tomar ventaja y almacenar los datos en una **matriz dispersa**.

- Una matriz con muchos ceros es dispersa (**sparse**)
- Lo opuesto a una matriz dispersa es una matriz densa (**dense**)
- Para optimizar memoria es posible guardar únicamente los datos que no son cero

Para ahorrar memoria se utilizan las matrices dispersas.


<img src="https://dziganto.github.io/assets/images/sparse_matrix.png?raw=true" alt="floating point" style="width: 30%"/>

Otro ejemplo de una matriz dispersa grande:

<img src="https://upload.wikimedia.org/wikipedia/commons/8/8a/Finite_element_sparse_matrix.png" tyle="width: 20%"/>

[Source](https://commons.wikimedia.org/w/index.php?curid=2245335)

Los formatos más comunes de almacenamiento de matrices dispersas son:
- coordinate-wise (scipy calls COO)
- compressed sparse row (CSR) *
- compressed sparse column (CSC)

[Aquí algumnos ejemplos](http://www.mathcs.emory.edu/~cheung/Courses/561/Syllabus/3-C/sparse.html)

[Aquí muchos otros formatos](http://www.cs.colostate.edu/~mcrob/toolbox/c++/sparseMatrix/sparse_matrix_compression.html)

### Nuestro CountVectorizer

In [25]:
Counter((movie_reviews.valid.x)[0].data)

Counter({2: 1,
         5: 32,
         21: 3,
         71: 1,
         189: 1,
         748: 1,
         288: 1,
         285: 1,
         63: 2,
         221: 1,
         666: 2,
         59: 1,
         13: 4,
         2705: 1,
         14: 6,
         2875: 1,
         11: 10,
         18: 2,
         358: 1,
         0: 32,
         77: 1,
         15: 6,
         478: 1,
         1833: 1,
         50: 3,
         9: 10,
         319: 1,
         6: 1,
         2743: 1,
         12: 1,
         115: 1,
         4126: 1,
         197: 2,
         1331: 1,
         25: 2,
         324: 1,
         10: 7,
         3963: 1,
         16: 4,
         74: 1,
         24: 3,
         2817: 1,
         5821: 1,
         2595: 1,
         710: 1,
         3429: 1,
         84: 1,
         149: 1,
         20: 1,
         26: 1,
         605: 1,
         378: 1,
         1057: 1,
         251: 1,
         258: 1,
         1346: 1,
         194: 1,
         239: 1,
         49: 1,
         27

In [26]:
movie_reviews.vocab.itos[71]

'very'

In [27]:
(movie_reviews.valid.x)[0]

Text xxbos xxmaj this very funny xxmaj british comedy shows what might happen if a section of xxmaj london , in this case xxmaj xxunk , were to xxunk itself independent from the rest of the xxup uk and its laws , xxunk & post - war xxunk . xxmaj merry xxunk is what would happen . 
 
  xxmaj the explosion of a wartime bomb leads to the xxunk of ancient xxunk which show that xxmaj xxunk was xxunk to the xxmaj xxunk of xxmaj xxunk xxunk ago , a small historical xxunk long since forgotten . xxmaj to the new xxmaj xxunk , however , this is an unexpected opportunity to live as they please , free from any xxunk from xxmaj xxunk . 
 
  xxmaj stanley xxmaj xxunk is excellent as the minor city xxunk who suddenly finds himself leading one of the world 's xxunk xxunk . xxmaj xxunk xxmaj margaret xxmaj xxunk is a delight as the history professor who sides with xxmaj xxunk . xxmaj others in the stand - out cast include xxmaj xxunk xxmaj xxunk , xxmaj paul xxmaj xxunk , xxmaj xxunk xxmaj xxunk , xxma

In [28]:
def get_term_doc_matrix(label_list, vocab_len):
    j_indices = []
    indptr = []
    values = []
    indptr.append(0)

    for i, doc in enumerate(label_list):
        feature_counter = Counter(doc.data)
        j_indices.extend(feature_counter.keys())
        values.extend(feature_counter.values())
        indptr.append(len(j_indices))
        
#     return (values, j_indices, indptr)

    return scipy.sparse.csr_matrix((values, j_indices, indptr),
                                   shape=(len(indptr) - 1, vocab_len),
                                   dtype=int)

In [29]:
%%time
val_term_doc = get_term_doc_matrix(movie_reviews.valid.x, len(movie_reviews.vocab.itos))

Wall time: 35.9 ms


In [30]:
%%time
trn_term_doc = get_term_doc_matrix(movie_reviews.train.x, len(movie_reviews.vocab.itos))

Wall time: 149 ms


In [31]:
trn_term_doc.shape

(800, 6008)

In [32]:
trn_term_doc[:,-10:]

<800x10 sparse matrix of type '<class 'numpy.intc'>'
	with 10 stored elements in Compressed Sparse Row format>

In [33]:
val_term_doc.shape

(200, 6008)

### Exploración adicional de datos:

Podemos convertir nuestra matriz de una representación dispersa a una representación fila columna.

In [34]:
movie_reviews.vocab.itos[-1:] #ultima palabra de la lista itos

['sollett']

In [35]:
val_term_doc.todense()[:10,:10]

matrix([[32,  0,  1,  0, ...,  1,  0,  0, 10],
        [ 9,  0,  1,  0, ...,  1,  0,  0,  7],
        [ 6,  0,  1,  0, ...,  0,  0,  0, 12],
        [78,  0,  1,  0, ...,  0,  0,  0, 44],
        ...,
        [ 8,  0,  1,  0, ...,  0,  0,  0,  8],
        [43,  0,  1,  0, ...,  8,  1,  0, 25],
        [ 7,  0,  1,  0, ...,  1,  0,  0,  9],
        [19,  0,  1,  0, ...,  2,  0,  0,  5]])

In [36]:
movie_reviews.vocab.itos[-1]

'sollett'

In [37]:
review = movie_reviews.valid.x[1]
review

Text xxbos i saw this movie once as a kid on the late - late show and fell in love with it . 
 
  xxmaj it took 30 + years , but i recently did find it on xxup dvd - it was n't cheap , either - in a xxunk that xxunk in war movies . xxmaj we watched it last night for the first time . xxmaj the audio was good , however it was grainy and had the trailers between xxunk . xxmaj even so , it was better than i remembered it . i was also impressed at how true it was to the play . 
 
  xxmaj the xxunk is around here xxunk . xxmaj if you 're xxunk in finding it , fire me a xxunk and i 'll see if i can get you the xxunk . xxunk

**Ejercicio:**

La palabra *late* aparece dos veces en esta crítica. Confirme que un valor de 2 es almanenado en la matriz de termino-documento.


#### Respuesta (Programe la respuesta aquí):

In [38]:
# Exercise: Confirm this


In [39]:
val_term_doc

<200x6008 sparse matrix of type '<class 'numpy.int32'>'
	with 27848 stored elements in Compressed Sparse Row format>

In [40]:
val_term_doc[1]

<1x6008 sparse matrix of type '<class 'numpy.intc'>'
	with 81 stored elements in Compressed Sparse Row format>

In [41]:
val_term_doc[1].sum()

144

La crítica tiene en total 81 tokens distintos con 144 en total.

In [42]:
review.data

array([  2,  19, 248,  21, ...,   9,   0,  10,   0], dtype=int64)

**Ejercicio 2:** Como convertir review.data en texto sin usar review.text?

#### Respuesta

In [43]:
# Exercise



**Ejercicio 3**: Confirme que review tiene exactamente 81 tokens

#### Answer

In [44]:
# Escriba aquí la respuesta


## ¿Por qué stoi tiene más elementos que itos?

In [45]:
movie_reviews.vocab.itos[1000:1005]

['state', 'street', 'impossible', 'clever', 'development']

`stoi` (string-to-int) is larger than `itos` (int-to-string).

In [46]:
len(movie_reviews.vocab.stoi) - len(movie_reviews.vocab.itos)

13154

Hay muchas palabras que mapean unknown:

In [47]:
unk = []
for word, num in movie_reviews.vocab.stoi.items():
    if num == 0:
        unk.append(word)

In [48]:
len(unk)

13155

In [49]:
unk[:5]

['xxunk', 'bleeping', 'pert', 'ticky', 'schtick']

## Naive Bayes

Definimos la relación logarítmica **log-count ratio** $r$ para cada palabra $f$:

$r = \log \frac{\text{ratio of feature $f$ in positive documents}}{\text{ratio of feature $f$ in negative documents}}$

Donde $f$ es el número de veces que un documento positivo/negativo tiene una característica dividida sobre el número de documentos positivos/negativos respectivamente.

In [50]:
movie_reviews.y.classes

['negative', 'positive']

In [51]:
x = trn_term_doc
y = movie_reviews.train.y
val_y = movie_reviews.valid.y

In [52]:
positive = y.c2i['positive'] #obtiene clase a index para positivo 1
negative = y.c2i['negative'] #obtain clase a index para negativo 0

In [53]:
?? x[y.items==positive].sum(0)

Object ` x[y.items==positive].sum(0)` not found.


In [54]:
p1 = np.squeeze(np.asarray(x[y.items==positive].sum(0))) 
p0 = np.squeeze(np.asarray(x[y.items==negative].sum(0)))

In [55]:
len(p1), len(p0)

(6008, 6008)

In [56]:
np.squeeze(np.asarray(x[y.items==negative].sum(0))) #conteos de una palabra totales en críticas negativas

array([7154,    0,  417,    0, ...,    0,    3,    3,    3], dtype=int32)

In [57]:
np.asarray(x[y.items==positive].sum(0)) #retorna una lista de listas

array([[6471,    0,  383,    0, ...,    3,    0,    0,    0]], dtype=int32)

In [58]:
np.squeeze(np.asarray(x[y.items==positive].sum(0))) #conteos de una palabra totales en críticas positivas, squeeze elimina esa dimensión adicional

array([6471,    0,  383,    0, ...,    3,    0,    0,    0], dtype=int32)

Para cada palabra en nuestro vocabulario estamos sumando en cuantas críticas positivas y en cuantas negativas están.

In [59]:
p1[:10]

array([ 6471,     0,   383,     0,     0, 10267,   674,    57,     0,  5260], dtype=int32)

In [60]:
v = movie_reviews.vocab

In [61]:
v.itos[0]

'xxunk'

In [62]:
v.itos[6004]

'coaxes'

### Usando los ratios para explorar datos

Se puede utilizar p0 y p1 para saber cuantas veces aparece una palabra dada en críticas positivas vs cuantas veces aparece en críticas negativas.

**Ejercicio**: Que tan a menudo aparece **loved** en críticas positivas vs. en críticas negativas... y **hated**?

#### Answer:

In [63]:
# Escriba aquí la relación para loved.


In [64]:
# Escriba aquí la relación para hated.


#### Obtener reviews positivos para la palabra hated

Es posible hacerlo:

In [65]:
v.stoi['hated']

1977

In [66]:
a = np.argwhere((x[:,1977] > 0))[:,0]
a

array([ 15,  49, 304, 351, 393, 612, 695, 773])

In [67]:
b = np.argwhere(y.items==positive)[:,0]
b

array([  1,   3,  10,  11, ..., 787, 789, 790, 797], dtype=int64)

In [68]:
set(a).intersection(set(b))

{393, 612, 695}

In [69]:
review = movie_reviews.train.x[695]
review.text

"xxbos xxmaj xxunk , yeah this episode is extremely underrated . \n \n  xxmaj even though there is a xxup lot of bad writing and acting at parts . i think the good over wins the bad . \n \n  i love the xxunk parts and the big ' twist ' at the end . i absolutely love that scene when xxmaj michelle xxunk xxmaj tony . xxmaj it 's actually one of my favorite scenes of xxmaj season 1 . \n \n  xxmaj for some reason , people have always hated the xxmaj xxunk episodes , yet i have always liked them . xxmaj they 're not the best , in terms of writing . but the theme really does interest me , \n \n  i 'm gon na give it a xxup three star , but if the writing were a little more consistent i 'd give it xxup four ."

#### Obtener críticas negativas con la palabra "loved"

Miremos algunas críticas negativas con la palabra loved

In [70]:
v.stoi['loved']

535

In [71]:
a = np.argwhere((x[:,535] > 0))[:,0]
a

array([  1,  15,  29,  69,  75,  79, 174, 185, 200, 205, 262, 296, 303, 333, 350, 351, 398, 407, 440, 489, 496, 528,
       538, 600, 602, 605, 627, 642, 657, 660, 700, 712, 729, 735, 755, 767, 785])

In [72]:
b = np.argwhere(y.items==negative)[:,0]
b

array([  0,   2,   4,   5, ..., 795, 796, 798, 799], dtype=int64)

In [73]:
set(a).intersection(set(b))

{15, 200, 205, 303, 351, 398, 600, 605, 642, 700, 729, 767}

In [74]:
review = movie_reviews.train.x[81]
review.text

"xxbos xxmaj this is n't one of xxmaj arbuckle 's or xxmaj keaton 's better films , that 's for sure . xxmaj fatty 's wife is tired of all his heavy drinking , so she takes him to a xxunk where a psychiatrist ( xxmaj keaton ) claims to have a guaranteed cure ! xxmaj well , once there , xxmaj arbuckle accidentally eats a xxunk and is taken to surgery . xxmaj then , he escapes and is chased about the place where he meets a cute girl who also wants to escape . xxmaj finally , despite xxunk chasing them about , they escape at which point it becomes apparent that the girl is crazy and xxmaj arbuckle is soon xxunk . xxmaj however , he xxunk and everything xxup after the surgery has all been a dream -- there was no sexy crazy girl and xxmaj dr. xxmaj keaton is n't as big an incompetent as he seemed in the dream . \n \n  a lack of humor is the biggest problem with the film . xxmaj sure , making fun of mentally ill people is pretty low , but in its day it was guaranteed laughs . i 'd laugh , to

## Aplicando Naive Bayes

In [75]:
p1 = np.squeeze(np.asarray(x[y.items==positive].sum(0)))
p0 = np.squeeze(np.asarray(x[y.items==negative].sum(0)))

In [76]:
pr1 = (p1+1) / ((y.items==positive).sum() + 1)
pr0 = (p0+1) / ((y.items==negative).sum() + 1)

In [77]:
r = np.log(pr1/pr0); r 

array([-0.015487,  0.084839,  0.      ,  0.084839, ...,  1.471133, -1.301455, -1.301455, -1.301455])

### Vocabulario asociado a críticas positivas o negativas

In [78]:
biggest = np.argpartition(r, -10)[-10:] #obtiene los 10 valores más grandes (críticas más positivas)
smallest = np.argpartition(r, 10)[:10]  #obtiene los valores más pequeños (críticas más negativas)

Palabras más positivas:

In [79]:
[v.itos[k] for k in biggest]

['sport',
 'davies',
 'gilliam',
 'fanfan',
 'biko',
 'felix',
 'noir',
 'jabba',
 'astaire',
 'jimmy']

In [80]:
np.argmax(trn_term_doc[:,v.stoi['biko']])

515

In [81]:
movie_reviews.train.x[515]

Text xxbos " xxmaj the xxmaj true xxmaj story xxmaj of xxmaj the xxmaj friendship xxmaj that xxmaj shook xxmaj south xxmaj africa xxmaj and xxmaj xxunk xxmaj the xxmaj world . " 
 
  xxmaj richard xxmaj attenborough , who directed " a xxmaj bridge xxmaj too xxmaj far " and " xxmaj gandhi " , wanted to bring the story of xxmaj steve xxmaj biko to life , and the journey and trouble that xxunk xxmaj donald xxmaj woods went through in order to get his story told . xxmaj the films uses xxmaj wood 's two books for it 's information and basis - " xxmaj biko " and " xxmaj asking for xxmaj trouble " . 
 
  xxmaj the film takes place in the late 1970 's , in xxmaj south xxmaj africa . xxmaj south xxmaj africa is in the grip of the terrible apartheid , which keeps the blacks separated from the whites and xxunk the whites as the superior race . xxmaj the blacks are forced to live in xxunk on the xxunk of the cities and xxunk , and they come under frequent xxunk by the police and the army . xxmaj w

Most negative words:

In [82]:
[v.itos[k] for k in smallest]

['worst',
 'crap',
 'crater',
 'porn',
 'disappointment',
 'dog',
 'vargas',
 'naschy',
 'fuqua',
 'soderbergh']

In [83]:
np.argmax(trn_term_doc[:,v.stoi['soderbergh']])

434

In [84]:
movie_reviews.train.x[434]

Text xxbos xxmaj now that xxmaj che(2008 ) has finished its relatively short xxmaj australian cinema run ( extremely limited xxunk screen in xxmaj xxunk , after xxunk ) , i can xxunk join both xxunk of " xxmaj at xxmaj the xxmaj movies " in taking xxmaj steven xxmaj soderbergh to task . 
 
  xxmaj it 's usually satisfying to watch a film director change his style / subject , but xxmaj soderbergh 's most recent stinker , xxmaj the xxmaj girlfriend xxmaj xxunk ) , was also missing a story , so narrative ( and editing ? ) seem to suddenly be xxmaj soderbergh 's main challenge . xxmaj strange , after xxunk years in the business . xxmaj he was probably never much good at narrative , just xxunk it well inside " edgy " projects . 
 
  xxmaj none of this excuses him this present , almost diabolical failure . xxmaj as xxmaj david xxmaj xxunk xxunk , " two parts of xxmaj che do n't ( even ) make a whole " . 
 
  xxmaj epic xxunk in name only , xxmaj che(2008 ) barely qualifies as a feature film 

In [85]:
trn_term_doc[:,v.stoi['soderbergh']]

<800x1 sparse matrix of type '<class 'numpy.intc'>'
	with 1 stored elements in Compressed Sparse Row format>

In [86]:
[v.itos[k] for k in smallest]

['worst',
 'crap',
 'crater',
 'porn',
 'disappointment',
 'dog',
 'vargas',
 'naschy',
 'fuqua',
 'soderbergh']

### Naive Bayes: continuación

In [87]:
(y.items==positive).mean(), (y.items==negative).mean()

(0.47875, 0.52125)

In [88]:
b = np.log((y.items==positive).mean() / (y.items==negative).mean()) #normaliza para encontrar la razón logarítmica del número promedio de críticas 

In [89]:
preds = (val_term_doc @ r + b) > 0

In [90]:
(preds == val_y.items).mean() #promedio

0.645

## Pasando al dataset completo 

Dado que funcionó con el data set de ejemplo pasamos a probar el dataset completo:

### Download data and process

In [91]:
path = untar_data(URLs.IMDB)
path.ls()

[WindowsPath('C:/Users/Arles/.fastai/data/imdb/imdb.vocab'),
 WindowsPath('C:/Users/Arles/.fastai/data/imdb/imdb_textlist_class'),
 WindowsPath('C:/Users/Arles/.fastai/data/imdb/lm_databunch'),
 WindowsPath('C:/Users/Arles/.fastai/data/imdb/models'),
 WindowsPath('C:/Users/Arles/.fastai/data/imdb/README'),
 WindowsPath('C:/Users/Arles/.fastai/data/imdb/test'),
 WindowsPath('C:/Users/Arles/.fastai/data/imdb/tmp_clas'),
 WindowsPath('C:/Users/Arles/.fastai/data/imdb/tmp_lm'),
 WindowsPath('C:/Users/Arles/.fastai/data/imdb/train'),
 WindowsPath('C:/Users/Arles/.fastai/data/imdb/unsup')]

In [92]:
(path/'train').ls()

[WindowsPath('C:/Users/Arles/.fastai/data/imdb/train/labeledBow.feat'),
 WindowsPath('C:/Users/Arles/.fastai/data/imdb/train/neg'),
 WindowsPath('C:/Users/Arles/.fastai/data/imdb/train/pos'),
 WindowsPath('C:/Users/Arles/.fastai/data/imdb/train/unsupBow.feat')]

In [93]:
reviews_full = (TextList.from_folder(path)
             #grab all the text files in path
             .split_by_folder(valid='test')
             #split by train and valid folder (that only keeps 'train' and 'test' so no need to filter)
             .label_from_folder(classes=['neg', 'pos']))
             #label them all with their folders

In [94]:
len(reviews_full.train), len(reviews_full.valid)

(25000, 25000)

We will store the vocab in a variable `v` since we will be using it frequently:

In [95]:
v = reviews_full.vocab

In [96]:
v.itos[100:110]

['bad',
 'people',
 'will',
 'other',
 'also',
 'into',
 'first',
 'because',
 'great',
 'how']

In [97]:
%%time
val_term_doc = get_term_doc_matrix(reviews_full.valid.x, len(reviews_full.vocab.itos))

Wall time: 4.63 s


In [98]:
%%time
trn_term_doc = get_term_doc_matrix(reviews_full.train.x, len(reviews_full.vocab.itos))

Wall time: 4.57 s


### Guardar y cargar la matriz de término-documento

Como el proceso es lento es una muy buena idea guardar la matriz y después cargarla.

In [99]:
scipy.sparse.save_npz("trn_term_doc.npz", trn_term_doc)

In [100]:
scipy.sparse.save_npz("val_term_doc.npz", val_term_doc)

Cuando se guarde debe añadirse a git.ignore en caso de manejar git.

Así se cargan los datos:

In [101]:
trn_term_doc = scipy.sparse.load_npz("trn_term_doc.npz")
val_term_doc = scipy.sparse.load_npz("val_term_doc.npz")

### Naive Bayes on full dataset

In [102]:
x=trn_term_doc
y=reviews_full.train.y

val_y = reviews_full.valid.y.items

In [103]:
x

<25000x38456 sparse matrix of type '<class 'numpy.int32'>'
	with 3716265 stored elements in Compressed Sparse Row format>

In [104]:
positive = y.c2i['pos']
negative = y.c2i['neg']

In [105]:
p0 = np.squeeze(np.asarray(x[y.items==negative].sum(0)))
p1 = np.squeeze(np.asarray(x[y.items==positive].sum(0)))

In [106]:
p1[:20]

array([ 28450,      0,  12500,      0,      0, 342619,  20464,   1338,      7, 173122, 138000, 143763,  89570,  83404,
        76828,  66715,  58510,  47896,  50177,  40451], dtype=int32)

### Exploración de datos: razones de una palabra en críticas positivas y negativas

Es posible obtener la razón entre el número de veces una palabra dada aparece en críticas negativas en positivas. Razones ( > 1) significan que la palabra indica un review negativo y valores (< 1) indican un review positivo.

In [107]:
def neg_pos_given_word(word):
    print(p0[v.stoi[word]]/p1[v.stoi[word]])

In [108]:
neg_pos_given_word('hated')

2.051546391752577


In [109]:
neg_pos_given_word('liked')

0.6424702058504875


In [110]:
neg_pos_given_word('loved')

0.3139963167587477


In [111]:
neg_pos_given_word('best')

0.48538961038961037


In [112]:
neg_pos_given_word('worst')

9.837301587301587


In [113]:
pr1 = (p1+1) / ((y.items==positive).sum() + 1)
pr0 = (p0+1) / ((y.items==negative).sum() + 1)

In [114]:
r = np.log(pr1/pr0)

In [115]:
r[v.stoi['hated']]

-0.7133498878774648

In [116]:
r[v.stoi['loved']]

1.1563661500586044

In [117]:
r[v.stoi['worst']]

-2.2826243504315076

In [118]:
r[v.stoi['best']]

0.7225576052173609

### Naive Bayes sobre el data set

In [119]:
negative = y.c2i['neg']
p0 = np.squeeze(np.asarray(x[y.items==negative].sum(0)))

Como tenemos un número igual de críticas positivas y negativas b es 0.

In [120]:
pr1 = (p1+1) / ((y.items==positive).sum() + 1)
pr0 = (p0+1) / ((y.items==negative).sum() + 1)

In [121]:
b = np.log((y.items==positive).mean() / (y.items==negative).mean()); b

0.0

In [122]:
preds = (val_term_doc @ r + b) > 0

Nuestro accuracy es de 80% para el dataset completo:

In [123]:
(preds == val_y).mean()

0.8084

### Naive Bayes Binarizado

Maybe it only matters whether a word is in the review or not (not the frequency of the word):

In [124]:
x=trn_term_doc.sign()
y=reviews_full.train.y

In [125]:
x.todense()[:10,:10]

matrix([[1, 0, 1, 0, ..., 1, 0, 0, 1],
        [1, 0, 1, 0, ..., 1, 0, 0, 1],
        [1, 0, 1, 0, ..., 0, 0, 0, 1],
        [0, 0, 1, 0, ..., 0, 1, 0, 1],
        ...,
        [1, 0, 1, 0, ..., 0, 0, 0, 1],
        [0, 0, 1, 0, ..., 1, 0, 0, 1],
        [0, 0, 1, 0, ..., 0, 0, 0, 1],
        [0, 0, 1, 0, ..., 1, 0, 0, 1]])

In [126]:
negative = y.c2i['neg']
positive = y.c2i['pos']

In [127]:
p1 = np.squeeze(np.asarray(x[y.items==positive].sum(0)))
p0 = np.squeeze(np.asarray(x[y.items==negative].sum(0)))

In [128]:
pr1 = (p1+1) / ((y.items==positive).sum() + 1)
pr0 = (p0+1) / ((y.items==negative).sum() + 1)

In [129]:
r = np.log(pr1/pr0)
b = np.log((y.items==positive).mean() / (y.items==negative).mean())

preds = (val_term_doc.sign() @ r + b) > 0

In [130]:
(preds==val_y).mean()

0.82924

## Regresión logística

Es posible ejecutar un modelo de regresión logística cuando las características son unigramas.

In [131]:
from sklearn.linear_model import LogisticRegression

In [132]:
m = LogisticRegression(C=0.1, dual=True, solver='liblinear')
m.fit(x, y.items.astype(int))
preds = m.predict(val_term_doc)
(preds==val_y).mean()

0.7704

Y la versión binaria:

In [133]:
m = LogisticRegression(C=0.1, dual=True, solver="liblinear")
m.fit(trn_term_doc.sign(), y.items.astype(int))
preds = m.predict(val_term_doc.sign())
(preds==val_y).mean()

0.88536

# Trigramas con Naive Bayes 

A continuación se muestra un modelo de regresión logística con Naive Bayes desarrollado [aquí](https://www.aclweb.org/anthology/P12-2018). Para cada documento computamos atributos binarizados como se describió en la parte superior pero esta vez utilizando bigramas y trigramas. Cada característica es un log-count ratio. Un modelo de regresión logística es entrenado para predecir sentimientos.

### n-gramas

Un n-grama es una secuencia continua de ítmes (donde un item es una letra, una sílaba o una palabra). Un 1-grama es un unigrama, un 2-gram es un bigrama y un 3-gram es un trigrama

En este caso son secuencias contínuas de palabras como: "the dog", "said that" "can't you".

In [134]:
path = untar_data(URLs.IMDB_SAMPLE)

In [135]:
movie_reviews = (TextList.from_csv(path, 'texts.csv', cols='text')
                .split_from_df(col=2)
                .label_from_df(cols=0))

In [136]:
v = movie_reviews.vocab.itos

In [137]:
vocab_len = len(v)

## Nuestros datos

### Creación de la matriz de entrenamiento

In [138]:
min_n=1
max_n=3

j_indices = []
indptr = []
values = []
indptr.append(0)
num_tokens = vocab_len

itongram = dict()
ngramtoi = dict()

Iteramos sobre todas las secuencias de palabras para crear los n-gramas:

In [139]:
for i, doc in enumerate(movie_reviews.train.x):
    feature_counter = Counter(doc.data)
    j_indices.extend(feature_counter.keys())
    values.extend(feature_counter.values())
    this_doc_ngrams = list()

    m = 0
    for n in range(min_n, max_n + 1):
        for k in range(vocab_len - n + 1):
            ngram = doc.data[k: k + n]
            if str(ngram) not in ngramtoi:
                if len(ngram)==1:
                    num = ngram[0]
                    ngramtoi[str(ngram)] = num
                    itongram[num] = ngram
                else:
                    ngramtoi[str(ngram)] = num_tokens
                    itongram[num_tokens] = ngram
                    num_tokens += 1
            this_doc_ngrams.append(ngramtoi[str(ngram)])
            m += 1

    ngram_counter = Counter(this_doc_ngrams)
    j_indices.extend(ngram_counter.keys())
    values.extend(ngram_counter.values())
    indptr.append(len(j_indices))

Es útil emplear diccionarios para convertir entre índices y strings (en este caso n-gramas). Tenemos dos estructuras generadas del código anterios `itongram` (index to n-gram) y `ngramtoi` (n-gram to index).

In [140]:
train_ngram_doc_matrix = scipy.sparse.csr_matrix((values, j_indices, indptr),
                                   shape=(len(indptr) - 1, len(ngramtoi)),
                                   dtype=int)

In [141]:
train_ngram_doc_matrix

<800x260374 sparse matrix of type '<class 'numpy.int32'>'
	with 678885 stored elements in Compressed Sparse Row format>

### Revisando los datos

In [142]:
len(ngramtoi), len(itongram)

(260374, 260374)

In [143]:
itongram[20005]

array([ 15,   9, 710], dtype=int64)

In [144]:
ngramtoi[str(np.array([15,   9,  710]))]

20005

In [145]:
itongram[100000]

array([1140,   33], dtype=int64)

In [146]:
v[15], v[9], v[710]

('to', 'the', 'leads')

In [147]:
itongram[100010]

array([5430,   10], dtype=int64)

In [148]:
v[5430], v[10]

('photographer', '.')

In [149]:
itongram[6116]

array([ 85, 191,  64], dtype=int64)

In [150]:
v[85], v[191], v[64]

('even', 'look', 'her')

In [151]:
itongram[80000]

array([2594,   14, 2618], dtype=int64)

In [152]:
v[2594], v[14], v[2618]

('loss', 'of', 'sleep')

### Cración de matriz de validación

In [153]:
j_indices = []
indptr = []
values = []
indptr.append(0)

for i, doc in enumerate(movie_reviews.valid.x):
    feature_counter = Counter(doc.data)
    j_indices.extend(feature_counter.keys())
    values.extend(feature_counter.values())
    this_doc_ngrams = list()

    m = 0
    for n in range(min_n, max_n + 1):
        for k in range(vocab_len - n + 1):
            ngram = doc.data[k: k + n]
            if str(ngram) in ngramtoi:
                this_doc_ngrams.append(ngramtoi[str(ngram)])
            m += 1

    ngram_counter = Counter(this_doc_ngrams)
    j_indices.extend(ngram_counter.keys())
    values.extend(ngram_counter.values())
    indptr.append(len(j_indices))

In [154]:
valid_ngram_doc_matrix = scipy.sparse.csr_matrix((values, j_indices, indptr),
                                   shape=(len(indptr) - 1, len(ngramtoi)),
                                   dtype=int)

In [155]:
valid_ngram_doc_matrix

<200x260374 sparse matrix of type '<class 'numpy.int32'>'
	with 121600 stored elements in Compressed Sparse Row format>

In [156]:
train_ngram_doc_matrix

<800x260374 sparse matrix of type '<class 'numpy.int32'>'
	with 678885 stored elements in Compressed Sparse Row format>

### Guardar matriz de n-gramas


In [157]:
scipy.sparse.save_npz("train_ngram_matrix.npz", train_ngram_doc_matrix)

In [158]:
scipy.sparse.save_npz("valid_ngram_matrix.npz", valid_ngram_doc_matrix)

In [159]:
with open('itongram.pickle', 'wb') as handle:
    pickle.dump(itongram, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
with open('ngramtoi.pickle', 'wb') as handle:
    pickle.dump(itongram, handle, protocol=pickle.HIGHEST_PROTOCOL)

### Cargar matriz de n-gramas

In [160]:
train_ngram_doc_matrix = scipy.sparse.load_npz("train_ngram_matrix.npz")
valid_ngram_doc_matrix = scipy.sparse.load_npz("valid_ngram_matrix.npz")

In [161]:
with open('itongram.pickle', 'rb') as handle:
    b = pickle.load(handle)
    
with open('ngramtoi.pickle', 'rb') as handle:
    b = pickle.load(handle)

## Naive Bayes

In [162]:
x=train_ngram_doc_matrix
y=movie_reviews.train.y

In [163]:
positive = y.c2i['positive']
negative = y.c2i['negative']

In [164]:
x

<800x260374 sparse matrix of type '<class 'numpy.int32'>'
	with 678885 stored elements in Compressed Sparse Row format>

In [165]:
k=260373

In [166]:
pos = (y.items == positive)[:k]
neg = (y.items == negative)[:k]

In [167]:
xx = x[:k]

In [168]:
valid_labels = [o == positive for o in movie_reviews.valid.y.items]

In [ ]:
p0 = np.squeeze(np.array(xx[neg].sum(0)))
p1 = np.squeeze(np.array(xx[pos].sum(0)))

In [ ]:
pr1 = (p1+1) / ((y.items==positive).sum() + 1)
pr0 = (p0+1) / ((y.items==negative).sum() + 1)

In [ ]:
r = np.log(pr1/pr0)

In [ ]:
b = np.log((y.items==positive).mean() / (y.items==negative).mean())

In [ ]:
b

In [ ]:
(y.items==positive).mean(), (y.items==negative).mean()

In [ ]:
pre_preds = valid_ngram_doc_matrix @ r.T + b

In [ ]:
pre_preds

In [ ]:
preds = pre_preds.T>0

In [ ]:
preds[:10]

In [ ]:
valid_labels = [o == positive for o in movie_reviews.valid.y.items]

In [ ]:
(preds == valid_labels).mean()

### Naive Bayes Binarizado

In [ ]:
trn_x_ngram_sgn = train_ngram_doc_matrix.sign()
val_x_ngram_sgn = valid_ngram_doc_matrix.sign()

In [ ]:
xx = trn_x_ngram_sgn[:k]

In [ ]:
p0 = np.squeeze(np.array(xx[neg].sum(0)))
p1 = np.squeeze(np.array(xx[pos].sum(0)))

In [ ]:
pr1 = (p1+1) / ((y.items==positive).sum() + 1)
pr0 = (p0+1) / ((y.items==negative).sum() + 1)

In [ ]:
r = np.log(pr1/pr0)
b = np.log((y.items==positive).mean() / (y.items==negative).mean())

pre_preds = val_x_ngram_sgn @ r.T + b
preds = pre_preds.T>0

In [ ]:
(preds==valid_labels).mean()

## Regresión Logística

Aquí ajustamos la regresión logística regularizada donde las características son los trigramas.

In [ ]:
from sklearn.linear_model import LogisticRegression

### usando CountVectorizer 

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
veczr = CountVectorizer(ngram_range=(1,3), preprocessor=noop, tokenizer=noop, max_features=800000)

In [ ]:
docs = movie_reviews.train.x

In [ ]:
train_words = [[docs.vocab.itos[o] for o in doc.data] for doc in movie_reviews.train.x]

In [ ]:
valid_words = [[docs.vocab.itos[o] for o in doc.data] for doc in movie_reviews.valid.x]

In [ ]:
%%time
train_ngram_doc = veczr.fit_transform(train_words)

In [ ]:
train_ngram_doc

In [ ]:
veczr.vocabulary_

In [ ]:
val_ngram_doc = veczr.transform(valid_words)

In [ ]:
val_ngram_doc

In [ ]:
vocab = veczr.get_feature_names()

In [ ]:
vocab[200000:200005]

#### Naive Bayes binarizado usando ngrams y CountVectorizer

In [ ]:
y=movie_reviews.train.y

C es la inversa de la fuerza de regularización; los valores más pequeños especifican una regularización más fuerte. Regularizado:

In [ ]:
m = LogisticRegression(C=0.1, solver="liblinear", dual=True)
m.fit(train_ngram_doc.sign(), y.items);

preds = m.predict(val_ngram_doc.sign())
(preds.T==valid_labels).mean()

No binarizado:

In [ ]:
m = LogisticRegression(C=0.1, solver="liblinear",dual=True)
m.fit(train_ngram_doc, y.items);

preds = m.predict(val_ngram_doc)
(preds.T==valid_labels).mean()

### Usando los ngrams, binarizado:

In [ ]:
m2 = LogisticRegression(C=0.1, solver="liblinear", dual=True)
m2.fit(trn_x_ngram_sgn, y.items)

In [ ]:
preds = m2.predict(val_x_ngram_sgn)
(preds.T==valid_labels).mean()

## References

* Baselines and Bigrams: Simple, Good Sentiment and Topic Classification. Sida Wang and Christopher D. Manning [pdf](https://www.aclweb.org/anthology/P12-2018)